# Proyecto Final: Sistema Multi-Agente (Versión Local)

Esta es la versión para **ejecución local** del proyecto. Incluye configuración para cargar la API Key desde un archivo `.env` o variables de entorno del sistema.

## Arquitectura "Doc Squad"
1.  **IngestAgent**: Gestión de archivos.
2.  **AnalystAgent**: Extracción de hechos técnicos.
3.  **TechWriterAgent**: Redacción de documentación.

## 1. Instalación y Configuración Local

### Paso 0: Crear Entorno Virtual (Recomendado)
Antes de instalar nada, es buena práctica aislar las dependencias:
```bash
# En tu terminal (dentro de la carpeta del proyecto):
python3 -m venv .venv
source .venv/bin/activate  # En Linux/Mac
# .venv\Scripts\activate # En Windows
```

Una vez activado, ejecuta la siguiente celda para instalar las librerías:

In [21]:
!pip install -q google-adk google-generativeai python-dotenv

In [22]:
import os
import time
import google.generativeai as genai
from google.adk.agents.llm_agent import Agent
from google.adk.runners import InMemoryRunner
from dotenv import load_dotenv

# --- CONFIGURACIÓN DE API KEY (LOCAL) ---
# 1. Intenta cargar desde un archivo .env en el mismo directorio
load_dotenv()

# 2. Busca la variable de entorno
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# 3. Si no existe, pide input manual
if not GOOGLE_API_KEY:
    print("⚠️ No se encontró la variable de entorno GOOGLE_API_KEY.")
    print("Consejo: Crea un archivo '.env' con el contenido: GOOGLE_API_KEY=tu_clave_aqui")
    GOOGLE_API_KEY = input("O introduce tu Google API Key aquí: ")

genai.configure(api_key=GOOGLE_API_KEY)
print(f"API Key configurada (Termina en: ...{GOOGLE_API_KEY[-4:] if GOOGLE_API_KEY else 'NONE'}).")

API Key configurada (Termina en: ...6wcY).


## 2. Herramientas del Sistema (Tools)

Definimos la herramienta de ingesta que usará el `IngestAgent`.

In [23]:
def ingest_multimedia_tool(file_path: str) -> str:
    """
    Sube un archivo a la API de Gemini y espera a que esté listo.
    Retorna el URI del archivo o un mensaje de error.
    """
    if not os.path.exists(file_path):
        return f"ERROR: El archivo {file_path} no existe en el sistema local."

    print(f"[System] Subiendo {file_path}...")
    try:
        file_upload = genai.upload_file(file_path)
        
        while file_upload.state.name == "PROCESSING":
            print("[System] Procesando...", end=".", flush=True)
            time.sleep(2)
            file_upload = genai.get_file(file_upload.name)

        if file_upload.state.name == "FAILED":
            return "ERROR: Falló el procesamiento en Gemini."

        print(f"\n[System] Archivo listo: {file_upload.uri}")
        return file_upload.uri
    except Exception as e:
        return f"ERROR CRÍTICO: {str(e)}"

## 3. Definición de Agentes (The Doc Squad)

Creamos los tres agentes y sus respectivos **Runners**.

In [24]:
# --- 1. INGEST AGENT ---
ingest_agent = Agent(
    model='gemini-flash-latest',
    name='IngestAgent',
    description="Gestiona la carga de archivos.",
    instruction="""
    Eres el IngestAgent. Tu único trabajo es recibir rutas de archivos locales y subirlos usando la herramienta 'ingest_multimedia_tool'.
    Una vez tengas el URI, devuélvelo confirmando que está listo para análisis.
    Si la herramienta falla, reporta el error claramente.
    """,
    tools=[ingest_multimedia_tool]
)
ingest_runner = InMemoryRunner(agent=ingest_agent)

# --- 2. ANALYST AGENT ---
analyst_agent = Agent(
    model='gemini-pro-latest',
    name='AnalystAgent',
    description="Analiza contenido técnico y extrae hechos.",
    instruction="""
    Eres el AnalystAgent, un Ingeniero de Sistemas Senior.
    Tu trabajo es recibir un URI de archivo (video, audio, imagen) y extraer TODOS los detalles técnicos.
    NO te preocupes por el formato bonito. Céntrate en la precisión.
    
    Debes extraer:
    - Comandos exactos ejecutados.
    - Mensajes de error o logs visibles.
    - Pasos de configuración realizados.
    - Direcciones IP, nombres de host, puertos.
    
    Salida esperada: Una lista de hechos técnicos crudos y cronológicos.
    """
)
analyst_runner = InMemoryRunner(agent=analyst_agent)

# --- 3. TECH WRITER AGENT ---
tech_writer_agent = Agent(
    model='gemini-pro-latest',
    name='TechWriterAgent',
    description="Genera documentación final.",
    instruction="""
    Eres el TechWriterAgent. Recibes una lista de hechos técnicos de un analista.
    Tu trabajo es convertir esos hechos en un documento profesional (Markdown).
    
    Estructura requerida:
    1. Título Descriptivo.
    2. Resumen Ejecutivo (1 párrafo).
    3. Prerrequisitos (si los hay).
    4. Procedimiento Paso a Paso (numerado).
    5. Solución de Problemas (si aplica).
    
    Usa bloques de código para comandos. Añade notas de advertencia (WARNING) si ves algo peligroso.
    Tu tono debe ser formal, claro y directo.
    """
)
tech_writer_runner = InMemoryRunner(agent=tech_writer_agent)

print("Squad inicializado: IngestAgent, AnalystAgent, TechWriterAgent.")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/m1txel/Escritorio/Proyecto_Kagle/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/m1txel/Escritorio/Proyecto_Kagle/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/m1txel/Escritorio/Proyecto_Kagle/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Squad inicializado: IngestAgent, AnalystAgent, TechWriterAgent.


## 4. Orquestación del Flujo (Async)

Usamos `run_debug` de los Runners para ejecutar los agentes de forma asíncrona.

In [25]:
async def run_documentation_pipeline(file_path: str, request_context: str = ""):
    print(f"--- INICIANDO PIPELINE PARA: {file_path} ---\n")
    
    # PASO 1: INGESTA
    print("🤖 IngestAgent: Trabajando...")
    # run_debug devuelve una lista de eventos. El último suele tener la respuesta final.
    ingest_events = await ingest_runner.run_debug(f"Sube y procesa el archivo: {file_path}")
    
    # Extraemos el texto del último evento (simplificación para demo)
    # En producción, inspeccionaríamos el tipo de evento.
    ingest_output = ""
    if ingest_events:
         # Intentamos buscar el evento que tenga texto del modelo
         for event in reversed(ingest_events):
             # Dependiendo de la versión de ADK, la estructura del evento varía.
             # Asumimos que si se imprime en consola, tiene representación textual.
             # Para este fix rápido, usaremos una cadena genérica si no podemos parsear,
             # pero run_debug imprime por stdout, así que lo verás.
             pass
    
    # Como run_debug imprime la salida, para pasarla al siguiente agente en esta demo simple,
    # vamos a asumir que el archivo fue procesado correctamente y pasaremos el path/contexto.
    # En una app real, usaríamos 'session_id' compartido o parsearíamos el evento 'ModelResponse'.
    print(f"✅ IngestAgent terminó. (Ver salida arriba)\n")
    
    # PASO 2: ANÁLISIS
    print("🤖 AnalystAgent: Analizando contenido...")
    analysis_prompt = f"El archivo {file_path} ha sido procesado. Contexto: {request_context}. Extrae los hechos técnicos."
    analysis_events = await analyst_runner.run_debug(analysis_prompt)
    print(f"✅ AnalystAgent terminó.\n")

    # PASO 3: REDACCIÓN
    print("🤖 TechWriterAgent: Redactando documento final...")
    # Aquí idealmente pasaríamos la salida del Analyst. 
    # Al usar run_debug sin memoria compartida entre runners, simulamos el paso.
    writer_prompt = f"Basado en el análisis técnico anterior (ver historial o asumir éxito del paso anterior para demo), genera el documento final para: {request_context}"
    writer_events = await tech_writer_runner.run_debug(writer_prompt)
    
    return "Proceso completado. Revisa la salida de consola para ver el documento generado por TechWriterAgent."

In [28]:
# -------------------------------------------------
# 1️⃣  Ruta del video que quieres procesar
# -------------------------------------------------
video_path = "/home/m1txel/Escritorio/Proyecto_Kagle/test_data/sample_video.mp4"

# -------------------------------------------------
# 2️⃣  Contexto opcional (puedes dejarlo vacío)
# -------------------------------------------------
request_context = "Tutorial de instalación básico"

# -------------------------------------------------
# 3️⃣  Ejecutamos el pipeline con el video
# -------------------------------------------------
# Como la función es async, usamos `await` dentro de una celda de Jupyter.
final_msg = await run_documentation_pipeline(
    video_path,
    request_context=request_context
)

# -------------------------------------------------
# 4️⃣  Mostramos el mensaje final (opcional)
# -------------------------------------------------
print(final_msg)

--- INICIANDO PIPELINE PARA: /home/m1txel/Escritorio/Proyecto_Kagle/test_data/sample_video.mp4 ---

🤖 IngestAgent: Trabajando...

 ### Continue session: debug_session_id

User > Sube y procesa el archivo: /home/m1txel/Escritorio/Proyecto_Kagle/test_data/sample_video.mp4
[System] Subiendo /home/m1txel/Escritorio/Proyecto_Kagle/test_data/sample_video.mp4...
[System] Procesando....
[System] Archivo listo: https://generativelanguage.googleapis.com/v1beta/files/nlwlyn0c639l


ServerError: 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}

## 5. Demo en Vivo

Ejecutamos el pipeline completo.

In [29]:
# Crear archivo dummy para la demo (si no existe)
dummy_filename = "demo_config_server.txt"
if not os.path.exists(dummy_filename):
    with open(dummy_filename, "w") as f:
        f.write("""
        [VIDEO TRANSCRIPT SIMULADO]
        Admin: Vamos a configurar el servidor web Apache.
        Admin: Primero, actualizamos los repositorios con 'sudo apt update'.
        Admin: Ahora instalamos apache con 'sudo apt install apache2 -y'.
        Admin: Verificamos el estado con 'systemctl status apache2'.
        Admin: Oh, veo un error en el log, el puerto 80 está ocupado.
        Admin: Vamos a cambiar el puerto en /etc/apache2/ports.conf a 8080.
        Admin: Reiniciamos el servicio 'sudo systemctl restart apache2'.
        Admin: Listo, funcionando en puerto 8080.
        """)

# Ejecutar Pipeline (con await)
final_msg = await run_documentation_pipeline(dummy_filename, request_context="Tutorial de instalación básico")
print(final_msg)

--- INICIANDO PIPELINE PARA: demo_config_server.txt ---

🤖 IngestAgent: Trabajando...

 ### Continue session: debug_session_id

User > Sube y procesa el archivo: demo_config_server.txt
[System] Subiendo demo_config_server.txt...

[System] Archivo listo: https://generativelanguage.googleapis.com/v1beta/files/csrywvjf3sp0


ServerError: 500 INTERNAL. {'error': {'code': 500, 'message': 'An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting', 'status': 'INTERNAL'}}